# Calculate thermal generator parameters

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

from gen_params import (calc_heat_rate,
                        calc_emis_rate)

In [2]:
# Set up directories
cwd = os.getcwd()
if 'dev' in cwd:
    parent_dir = os.path.dirname(cwd)
    data_dir = os.path.join(parent_dir, 'data')
else:
    data_dir = os.path.join(cwd, 'data')

grid_data_dir = os.path.join(data_dir, 'grid', '2018Baseline')
if not os.path.exists(grid_data_dir):
    raise FileNotFoundError('Grid data directory not found.')

thermal_data_dir = os.path.join(data_dir, 'thermal')
if not os.path.exists(thermal_data_dir):
    raise FileNotFoundError('Thermal data directory not found.')

print('Grid data directory: {}'.format(grid_data_dir))
print('Thermal data directory: {}'.format(thermal_data_dir))

Grid data directory: /mnt/Bo_HDD/NYgrid-python/data/grid/2018Baseline
Thermal data directory: /mnt/Bo_HDD/NYgrid-python/data/thermal


# Read EPA CEMS historical generation and emissions data

In [3]:
cems_data_dir = os.path.join(thermal_data_dir, 'cems_2018')
df_list = list()
for mo in range(1, 13):
    df = pd.read_csv(os.path.join(cems_data_dir, f'2018ny{str(mo).zfill(2)}.csv'),
                     low_memory=False)
    df_list.append(df)

# Concatenate all the dataframes
cems_df = pd.concat(df_list, axis=0, ignore_index=True)

# Rename columns
cems_df = cems_df.rename(columns={
    'ORISPL_CODE': 'Plant_ID',
    'UNITID': 'Unit_ID',
})

# Add time columns from OP_DATE and OP_HOUR
cems_df['Time'] = pd.to_datetime(
    cems_df['OP_DATE']) + pd.to_timedelta(cems_df['OP_HOUR'], unit='h')
# cems_df['Month'] = cems_df['Time'].dt.month

# Drop columns that are not needed
cems_df = cems_df.drop(columns=['STATE', 'FAC_ID', 'UNIT_ID', 'OP_DATE', 'OP_HOUR',
                                'SO2_MASS_MEASURE_FLG', 'SO2_RATE_MEASURE_FLG',
                                'NOX_MASS_MEASURE_FLG', 'NOX_RATE_MEASURE_FLG',
                                'CO2_MASS_MEASURE_FLG', 'CO2_RATE_MEASURE_FLG',
                                'SLOAD (1000lb/hr)'])

# Read NYISO and CEMS generator matching table

In [4]:
gen_combiner = pd.read_excel(os.path.join(thermal_data_dir, '2018_nyca_thermal.xlsx'),
                             sheet_name='matched_with_id')
gen_combiner = gen_combiner.rename(columns={
    '   Station        Unit': 'NYISO_Name',
})

# Map unit type GT, JE to CT
gen_combiner['Unit Type'] = gen_combiner['Unit Type'].replace(
    {'GT': 'CT', 'JE': 'CT'})
gen_combiner['Fuel Type Secondary'] = gen_combiner['Fuel Type Secondary'].astype(
    str)
gen_combiner

,NYISO_Name,Zone,PTID,Name Plate Rating (MW),Dual Fuel,Unit Type,Fuel Type Primary,Fuel Type Secondary,2017 Net Energy (GWh),Note,Combinned,CAMD_Facility_Name,CAMD_Plant_ID,CAMD_Unit_ID,CAMD_Generator_ID,CAMD_Nameplate_Capacity,CAMD_Fuel_Type,EIA_Latitude,EIA_Longitude,ID
0,Danskammer 1,G,23586,72.0,YES,ST,NG,FO6,0.93040,NaN,NaN,Danskammer Generating Station,2480,1,1,72.0,Pipeline Natural Gas,41.571247,-73.974981,"(2480,1)"
1,Danskammer 2,G,23589,73.5,YES,ST,NG,FO6,0.90830,NaN,NaN,Danskammer Generating Station,2480,2,2,73.5,Pipeline Natural Gas,41.571247,-73.974981,"(2480,2)"
2,Danskammer 3,G,23590,147.1,NaN,ST,NG,nan,2.06448,NaN,NaN,Danskammer Generating Station,2480,3,3,147.1,Pipeline Natural Gas,41.571247,-73.974981,"(2480,3)"
3,Danskammer 4,G,23591,239.4,NaN,ST,NG,nan,5.05390,NaN,NaN,Danskammer Generating Station,2480,4,4,239.4,Pipeline Natural Gas,41.571247,-73.974981,"(2480,4)"
4,Arthur Kill ST 2,J,23512,376.2,NaN,ST,NG,nan,563.62800,NaN,NaN,Arthur Kill,2490,20,2,376.2,Pipeline Natural Gas,40.591564,-74.200035,"(2490,20)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Bayonne EC CTG4,J,323685,64.0,YES,CT,NG,KER,67.92100,NJ,NaN,Bayonne Energy Center,56964,GT4,GT4,64.0,Pipeline Natural Gas,40.652834,-74.091550,"(56964,GT4)"
356,Bayonne EC CTG5,J,323686,64.0,YES,CT,NG,KER,89.64800,NJ,NaN,Bayonne Energy Center,56964,GT5,GT5,64.0,Pipeline Natural Gas,40.652834,-74.091550,"(56964,GT5)"
357,Bayonne EC CTG6,J,323687,64.0,YES,CT,NG,KER,72.82700,NJ,NaN,Bayonne Energy Center,56964,GT6,GT6,64.0,Pipeline Natural Gas,40.652834,-74.091550,"(56964,GT6)"
358,Bayonne EC CTG7,J,323688,64.0,YES,CT,NG,KER,83.81700,NJ,NaN,Bayonne Energy Center,56964,GT7,GT7,64.0,Pipeline Natural Gas,40.652834,-74.091550,"(56964,GT7)"


In [5]:
os.makedirs('figures/time_series', exist_ok=True)
os.makedirs('figures/heat_rate', exist_ok=True)
os.makedirs('figures/nox_rate', exist_ok=True)
os.makedirs('figures/co2_rate', exist_ok=True)
os.makedirs('figures/so2_rate', exist_ok=True)

gen_params = dict()

# for ii in range(gen_combiner.shape[0]):
for ii in range(5):

    gen_info = gen_combiner.iloc[ii]

    if gen_info['ID'] not in gen_params:

        gen_params[gen_info['ID']] = dict()

        unit_df = cems_df[(cems_df['Plant_ID'] == gen_info['CAMD_Plant_ID'])
                          & (cems_df['Unit_ID'] == gen_info['CAMD_Unit_ID'])]
        unit_df = unit_df.set_index('Time').sort_index()
        # print(f'Number of rows: {unit_df.shape[0]}')

        # Calculate hourly ramp
        unit_df['RAMP (MW/hour)'] = unit_df['GLOAD (MW)'].diff()

        # Filter out rows with zero values
        unit_df_nonzero = unit_df[(unit_df['HEAT_INPUT (mmBtu)'] > 0) & (
            unit_df['GLOAD (MW)'] > 0)]
        unit_df_nonzero = unit_df_nonzero.drop(columns=['FACILITY_NAME',
                                                        'Plant_ID', 'Unit_ID'])
        # print(f'Number of rows with nonzero values: {unit_df_nonzero.shape[0]}')

        if unit_df_nonzero.shape[0] > 0:

            fig_name_sfx = '_'.join([gen_info["ID"], gen_info["NYISO_Name"],
                                     gen_info["PTID"].astype(str),
                                     gen_info["Unit Type"], gen_info["Fuel Type Primary"],
                                     gen_info["Fuel Type Secondary"]])

            # Time series plot
            df_plot = unit_df[['OP_TIME', 'GLOAD (MW)', 'SO2_MASS (lbs)', 'NOX_MASS (lbs)',
                               'CO2_MASS (tons)', 'HEAT_INPUT (mmBtu)']]
            fig, ax = plt.subplots(6, 1, figsize=(8, 6), sharex=True,
                                   layout='constrained')
            for i in range(6):
                ax[i].plot(df_plot.iloc[:, i])
                ax[i].set_title(df_plot.columns[i])

            title = (fig_name_sfx + '_' + gen_info["Unit Type"]
                     + '_' + gen_info["Fuel Type Primary"]
                     + '_' + gen_info["Fuel Type Secondary"])
            fig.suptitle(title)
            fig.savefig(f'figures/time_series/time_series_{fig_name_sfx}.png')

            # Calculate ramp rate
            raise_rr_ratio = (unit_df_nonzero['RAMP (MW/hour)'].max() 
                              / gen_info['CAMD_Nameplate_Capacity'])
            lower_rr_ratio = np.abs(unit_df_nonzero['RAMP (MW/hour)'].min() 
                                    / gen_info['CAMD_Nameplate_Capacity'])
            raise_rr_ratio = raise_rr_ratio if raise_rr_ratio <= 1 else 1
            lower_rr_ratio = lower_rr_ratio if lower_rr_ratio <= 1 else 1

            gen_params[gen_info['ID']]['max_mw'] = gen_info['CAMD_Nameplate_Capacity']
            gen_params[gen_info['ID']]['raise_rr_ratio'] = raise_rr_ratio
            gen_params[gen_info['ID']]['lower_rr_ratio'] = lower_rr_ratio       

            # Heat rate plot
            heat_1, heat_0, heat_r2, econ_min, gen_sum, heat_sum, fig, ax = \
                calc_heat_rate(data=unit_df_nonzero,
                               gen_info=gen_info,
                               x_name='GLOAD (MW)',
                               y_name='HEAT_INPUT (mmBtu)')
            gen_params[gen_info['ID']]['heat_1'] = heat_1
            gen_params[gen_info['ID']]['heat_0'] = heat_0
            gen_params[gen_info['ID']]['heat_r2'] = heat_r2
            gen_params[gen_info['ID']]['econ_min'] = econ_min
            gen_params[gen_info['ID']]['gen_sum'] = gen_sum
            gen_params[gen_info['ID']]['heat_sum'] = heat_sum
            if fig is not None:
                fig.savefig(f'figures/heat_rate/heat_rate_{fig_name_sfx}.png')

            # NOx emission rate plot
            nox_1, nox_0, nox_r2, nox_avg_rate, nox_sum, fig, ax = \
                calc_emis_rate(data=unit_df_nonzero,
                               gen_info=gen_info,
                               x_name='HEAT_INPUT (mmBtu)',
                               y_name='NOX_MASS (lbs)',
                               rate_name='NOX_RATE (lbs/mmBtu)')
            gen_params[gen_info['ID']]['nox_1'] = nox_1
            gen_params[gen_info['ID']]['nox_0'] = nox_0
            gen_params[gen_info['ID']]['nox_r2'] = nox_r2
            gen_params[gen_info['ID']]['nox_sum'] = nox_sum
            if fig is not None:
                fig.savefig(f'figures/nox_rate/nox_rate_{fig_name_sfx}.png')

            # CO2 emission rate plot
            co2_1, co2_0, co2_r2, co2_avg_rate, co2_sum, fig, ax = \
                calc_emis_rate(data=unit_df_nonzero,
                               gen_info=gen_info,
                               x_name='HEAT_INPUT (mmBtu)',
                               y_name='CO2_MASS (tons)',
                               rate_name='CO2_RATE (tons/mmBtu)')
            gen_params[gen_info['ID']]['co2_1'] = co2_1
            gen_params[gen_info['ID']]['co2_0'] = co2_0
            gen_params[gen_info['ID']]['co2_r2'] = co2_r2
            gen_params[gen_info['ID']]['co2_sum'] = co2_sum
            if fig is not None:
                fig.savefig(f'figures/co2_rate/co2_rate_{fig_name_sfx}.png')

            # SO2 emission rate plot
            so2_1, so2_0, so2_r2, so2_avg_rate, so2_sum, fig, ax = \
                calc_emis_rate(data=unit_df_nonzero,
                               gen_info=gen_info,
                               x_name='HEAT_INPUT (mmBtu)',
                               y_name='SO2_MASS (lbs)',
                               rate_name='SO2_RATE (lbs/mmBtu)')
            gen_params[gen_info['ID']]['so2_1'] = so2_1
            gen_params[gen_info['ID']]['so2_0'] = so2_0
            gen_params[gen_info['ID']]['so2_r2'] = so2_r2
            if fig is not None:
                fig.savefig(f'figures/so2_rate/so2_rate_{fig_name_sfx}.png')

            plt.close('all')

        else:
            print(f'No data for {fig_name_sfx}.')
            gen_params[gen_info['ID']] = None

    else:
        print(f'Data for {gen_info["ID"]} already processed.')

/mnt/Bo_HDD/NYgrid-python/dev/gen_params.py:169: UserWarning: Heat rate 6.06 too low for ST
  warn(f'Heat rate {heat_rate:.2f} too low for ST', UserWarning)


In [10]:
# Save the parameters to a file
gen_params_df = pd.DataFrame(gen_params).T
gen_params_df.to_csv(os.path.join('gen_params.csv'))